<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/REU_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pcm = None


# Turn pcm into a graph using Pytorch Geometric

check_nodes = pcm.shape[0]
variable_nodes = pcm.shape[1]

edges = None

# Have the edges describe relations between cn and vn
  # Also have other edges for relation between nodes that are not necessarily directly linked for Transformer model

# Message Passing embeddings

#


NameError: name 'Null' is not defined